<a href="https://colab.research.google.com/github/RaisaAnika/podcast_summary_openai/blob/main/project_buildingaiproduct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 1 - Podcast Transcription and Information Extraction

`Library Installations`

In [ ]:
!pip install feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=32dfb3cedebfee10e9e4ee3f2106f610ebd98f127797635d382837763aa8c546
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.7 MB/s eta 0:00:00


Whisper -- Speech to Text Model

In [ ]:
!pip install git+https://github.com/openai/whisper.git  -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.6 MB/s eta 0:00:00


In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=48a9fd7d94533d7da5019ba16077deb49a4e9d79838f317135b144df2f20d6db
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
!pip install gTTS


In [ ]:
#@title Library Imports
import openai
from getpass import getpass

import feedparser


import pathlib
import whisper

import tiktoken

import re
import spacy

import json

import wikipedia

from gtts import gTTS
import IPython.display as ipd

In [ ]:
model_path = pathlib.Path("/content/podcast/medium.pt")
if model_path.exists():
  print ("Model has been downloaded, no re-download necessary")
else:
  print ("Starting download of Whisper Model")
  whisper._download(whisper._MODELS["medium"], '/content/podcast/', False)

Starting download of Whisper Model


100%|█████████████████████████████████████| 1.42G/1.42G [00:34<00:00, 44.3MiB/s]


In [ ]:
#@title openAPI

openai.api_key = getpass('Enter the OpenAI API Key in the cell ')

Enter the OpenAI API Key in the cell ··········


In [ ]:
#confirm that the API key works by listing all the OpenAI models
models = openai.Model.list()
for model in models["data"]:
  print (model["root"])

gpt-3.5-turbo-16k-0613
text-davinci-001
text-search-curie-query-001
davinci
babbage
text-babbage-001
curie-instruct-beta
text-davinci-003
davinci-similarity
code-davinci-edit-001
text-similarity-curie-001
ada-code-search-text
text-search-ada-query-001
babbage-search-query
ada-similarity
text-curie-001
text-search-ada-doc-001
text-search-babbage-query-001
code-search-ada-code-001
curie-search-document
text-search-davinci-query-001
text-search-curie-doc-001
babbage-search-document
babbage-code-search-text
text-embedding-ada-002
davinci-instruct-beta
davinci-search-query
text-similarity-babbage-001
text-davinci-002
code-search-babbage-text-001
text-search-davinci-doc-001
code-search-ada-text-001
ada-search-query
gpt-3.5-turbo
text-similarity-ada-001
ada-code-search-code
whisper-1
text-davinci-edit-001
gpt-3.5-turbo-0613
davinci-search-document
curie-search-query
babbage-similarity
ada
ada-search-document
text-ada-001
text-similarity-davinci-001
gpt-3.5-turbo-16k
curie-similarity
babbage-c

In [ ]:

podcast_feed_url = "https://feeds.megaphone.fm/LI3617121267"#"YOUR_PODCAST_RSS_FEED_URL"
podcast_feed = feedparser.parse(podcast_feed_url)


In [ ]:
# Get the title of the podcast feed
podcast_title = podcast_feed.feed.title
print ("The number of podcast entries for ",podcast_title, "is ", len(podcast_feed.entries))


The number of podcast entries for  Big Technology Podcast is  210


In [ ]:
for item in podcast_feed.entries[0].links:
  if (item['type'] == 'audio/mpeg'):
    episode_url = item.href
!wget -O 'podcast_episode.mp3' {episode_url}

--2023-08-19 08:08:55--  https://traffic.megaphone.fm/LI4754407970.mp3?updated=1692402932
Resolving traffic.megaphone.fm (traffic.megaphone.fm)... 34.72.38.16
Connecting to traffic.megaphone.fm (traffic.megaphone.fm)|34.72.38.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dcs.megaphone.fm/LI4754407970.mp3?key=12b055a439519b0f35721b65c003b567&request_event_id=4e53ce0f-af7b-4cce-9edc-add568f9073a [following]
--2023-08-19 08:08:56--  https://dcs.megaphone.fm/LI4754407970.mp3?key=12b055a439519b0f35721b65c003b567&request_event_id=4e53ce0f-af7b-4cce-9edc-add568f9073a
Resolving dcs.megaphone.fm (dcs.megaphone.fm)... 205.185.216.42, 205.185.216.10
Connecting to dcs.megaphone.fm (dcs.megaphone.fm)|205.185.216.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33793930 (32M) [audio/mpeg]
Saving to: ‘podcast_episode.mp3’

podcast_episode.mp3 100%[===================>]  32.23M  8.06MB/s    in 4.0s    

2023-08-19 08:09:02 (8.06 M

In [ ]:
#@title Load whisper model from saved location
model = whisper.load_model('medium', device='cuda', download_root='/content/podcast/')

In [ ]:
%%time
result = model.transcribe("/content/podcast_episode.mp3")

CPU times: user 8min, sys: 2.66 s, total: 8min 3s
Wall time: 8min 39s


In [ ]:
# Check the transcription happened correctly by peeking into the first 1000 characters
podcast_transcript = result['text']
result['text'][:1000]

" All that cage match build up and nothing to show for it. Goldman Sachs is in a rut just as its partnership with Apple is taking off. And have we been overplaying the impact of social media algorithms all this time? All that and more coming up right after this. Hi, I'm Ryan Ruzlanski, the CEO of LinkedIn. Listen to my new podcast, The Path. Discover how today's greatest leaders shape their unique career paths and the lessons they learned along the way. Tune in to new episodes weekly wherever you enjoy your podcasts. Welcome to Big Technology Podcast Friday edition where we break down the news in our traditional cool headed and nuanced format. We have a number of very interesting stories that we plan to go deep on this week. And we're so glad you're here with us. Speaking of which, returning from his vacation. Here with us today is Ron Johnroy. Ron John, welcome back. It is great to be back. I'm excited to talk. Let's get to the cage match soon because I have been waiting probably thre

In [ ]:
podcast_transcript

" All that cage match build up and nothing to show for it. Goldman Sachs is in a rut just as its partnership with Apple is taking off. And have we been overplaying the impact of social media algorithms all this time? All that and more coming up right after this. Hi, I'm Ryan Ruzlanski, the CEO of LinkedIn. Listen to my new podcast, The Path. Discover how today's greatest leaders shape their unique career paths and the lessons they learned along the way. Tune in to new episodes weekly wherever you enjoy your podcasts. Welcome to Big Technology Podcast Friday edition where we break down the news in our traditional cool headed and nuanced format. We have a number of very interesting stories that we plan to go deep on this week. And we're so glad you're here with us. Speaking of which, returning from his vacation. Here with us today is Ron Johnroy. Ron John, welcome back. It is great to be back. I'm excited to talk. Let's get to the cage match soon because I have been waiting probably thre

In [ ]:
# podcast_transcript = """
#  All that cage match build up and nothing to show for it. Goldman Sachs is in a rut just as its partnership with Apple is taking off. And have we been overplaying the impact of social media algorithms all this time? All that and more coming up right after this. Hi, I'm Ryan Ruzlanski, the CEO of LinkedIn. Listen to my new podcast, The Path. Discover how today's greatest leaders shape their unique career paths and the lessons they learned along the way. Tune in to new episodes weekly wherever you enjoy your podcasts. Welcome to Big Technology Podcast Friday edition where we break down the news in our traditional cool headed and nuanced format. We have a number of very interesting stories that we plan to go deep on this week. And we're so glad you're here with us. Speaking of which, returning from his vacation. Here with us today is Ron Johnroy. Ron John, welcome back. It is great to be back. I'm excited to talk. Let's get to the cage match soon because I have been waiting probably three weeks to talk about this. So fresh off the plane from Taiwan, he wants to talk about the cage match. We're gonna talk about your reflections on Taiwan at the very end of this show. But first, let's talk about the cage match. So it turns out that this thing is now likely not going to happen at all. Elon Musk has been playing some games. Zuck basically said he's not serious and is now saying I'm going to move on. Ron John, you said you've been wanting to speak about this for a while. I think most people are talked out on the cage match. Why are you still eager to discuss it? I am eager to discuss it. Because if you remember, and if listeners remember, over the last few months, every time we do bring it up, I definitely have to shake my head and say I don't want to talk about it. We shouldn't be talking about it. The reason I'm now ready to talk about it is we are all getting to see another beautiful cycle of Elon Musk. Everyone is a part of it now. Mark Zuckerberg is a part of it. Walter Isaacson, which we'll get to in his new book is a part of it. This whole idea of and we've talked about it a lot, you know, is Elon Musk is trying to take test out different messaging, and then run with the ones that he knows will stick and clearly everyone got hyped up around this cage match. But I think what we're starting to see is him releasing private messages, text messages with Mark Zuckerberg, Walter Isaacson sharing them, then Elon Musk tweeting about, you know, self driving, leading a self driving horde of cars to Mark Zuckerberg's front door. I mean, we are definitely we've escalated and moved past the point of fun to discomfort, I think for everyone. And that's actually my favorite part to watch this whole cycle because I this is always where it ends. This is always where not well, not always because and this is kind of what I always think about with these Musk cycles we've gone through over the past couple years is that, you know, yes, like he's outlandish and has these wild ideas and doesn't fall through and a lot of things, but he's still running SpaceX and Tesla and doing a pretty good job with those companies. And maybe he has some others on the way Neuralink and, you know, there was this article in the New York Times about how like, oh, it's such a big problem that Elon Musk runs Starlink because now the Ukrainians can only rely on him. And it's like if he didn't build Starlink, the Ukrainians wouldn't have this service. So what are we complaining about here? So I always wonder what these type of situations is it is it is this part of this this whole personality part of what enabled Elon to sort of press the limits and think about things that he wouldn't that that most entrepreneurs are and governments for that matter, wouldn't otherwise and be like the second side of this is, is some of the things that we're seeing with him and Twitter, the same stuff that got him to where he is, or is it sort of that that inflection point of like, oh, god, now Elon might be in trouble? What do you think? I think it is exactly the same type of thing that got him to where it is because this this cage match saga, to me has always essentially been capital raising arbitrage for Elon Musk. It's he has been able to build. Yeah. Okay, he has been able to build these businesses. Think about Tesla, its growth trajectory, it becoming at one point, you know, a trillion dollar company him becoming at times the world's richest man are all like able to be done because of the narrative he has created around himself and the attention he commands. And I think that was his biggest insight is understanding that the more attention he's able to bring to himself, and the more he can kind of just command the entire state of conversation in the world. That's what allows him to raise the money that he needs or run Tesla in the way he wants to run it or build a new factory and make heads of state want to work with him and give him all the incentive incentives and subsidies possible again, like every head of state fawning over bringing Tesla to their country and bragging about when it does appear. That's okay, but because it's called the personality and how is this cage match going to pay off for him then? Oh, I think. Okay, where this all ends up, I'm not trying to predict just yet. But I think in terms of other than there won't be a cage match, I think, but he's had so many of these moments. I think this is just, you know, his profile only continues to grow. But this is the way he's been acting ever since again, for me, and I've written about this many times, I was a big Elon Musk fanboy in the early days of called the middle days of Tesla in the early 2010s. It was really the 420 tweet back it was at 2018, when he said he was going to take Tesla private and starting to see just how reckless a lot of the behavior was. And it was and again, think about that. Again, like tweeting that he's going to take the company private, getting a slap on the hand from the SEC, and then bringing more attention to self and then turning Tesla into a trillion dollar company based on just an insane stock valuations like he has been able to do all of that over and over again. So the cage match, it's a bet for him, I'm sure that maybe I'll get away with get away with it again. Okay, but here's the question then if this is the same behavior that's enabled him to build, which I think what I think most people would agree are pretty impressive companies, then what's the problem? He's just doing it again. Oh, because it's gonna run out at some point. At some point. Come on. This is no no, I'm running since the 420 tweet and Tesla and Tesla's building more cars than ever before. SpaceX has been able to like, basically own lower orbit and get those starlink satellites in so these so with this, I'm just kind of talking it out with you, right? I want to understand but with all this attention, he's actually been able to use that for good. So it's it's sort of like, is this is this really a problem? No, I think we're going to see again, the idea of like the narrative SpaceX, Tesla, shooting rockets to Mars and, and you know, electrifying the entire motor vehicle industry, that shine already you're starting to see cracks. I remember I was reading I think it was Felix Salmon had a piece recently around a lot of the new data and narratives that are coming out around was Tesla or has it been overall a net positive for electrification of the automotive industry? We take that as an unadulterated truth, right? Like everyone's like, Yeah, Tesla without it, nothing happens. But could hybrids have been a better way to approach this could smaller vehicles and making range the central part of the entire discussion has distracted us and potentially made us build batteries that aren't the optimal way and build transportation infrastructure that isn't the optimal one. So I think there's going to be a lot more questioning of that narrative because it's that narrative and that unquestioning acceptance of it that has allowed him to build these companies in that way. And I think I mean, I don't want to call the turning point ever with Elon Musk, because you never know. But again, the cage match, I think is exposing that kind of behavior to more and more people in more and more ways. Well, I think we'll just have to wait and see on this one. I mean, I think that he on this this entire chapter has not been, let's say impressive. But if it's gonna take the same path that it's taken with Tesla and SpaceX, and he's just going to command more capital to build more companies, actually, you know, he's figured out the system well, and he has it over the last four to five years, he figured out the system better than anyone else. I definitely agree with that. And the person who's gonna speaking to figure out the system, right, the person who's going to go and write a biography of him as well, none other than Walter Isaacson, he have the Steve Jobs biography. And it has been interesting watching Walter Isaacson, who's this like luminary author, go out and speak about Elon during this entire process, because he's almost become, he has become Elon's de facto PR person. Elon doesn't, you know, have a lot of or any public relations officials pretty much. I mean, there are there is one guy at Twitter who keeps emailing out this stuff from Linda Yaccarino. So I'm thinking like, okay, maybe PR and function if not in name. But Elon doesn't really use PR people. But Walter Isaacson has effectively been functioning as that person. And he did tweet out the private messages between Musk and Zuckerberg about the fight and in a way that served Musk's interest. There was some debate about that, right, where people were like, well, he should not have, you know, tweeted those or shared those texts. And I'm like, come on, you're not your reporter, you share the text with like the appropriate context. But just the total tone of Isaacson towards Musk has been that of as publicist and this book that he has, that's going to come out about Elon, which is coming out in about a month, might put his I mean, it might put his credibility on the line. Yeah, I think I really am curious how his how this plays out for Walter Isaacson's legacy as a venerated biographer, like, I mean, the idea that, again, 30 days must feel like an eternity for Walter Isaacson right now, because what will Elon Musk do in those within those 30 days? And what could that mean for when that book comes out, and it's glowing, and it's praising and it's saying that this is and again, if anyone who's read the biographies of Steve Jobs early early in order to Benji, which are excellent, they again, they paint a very, very clear picture of people who are changing the world for good, it might be a little complex. But overall, as you said, it's not quite fawning. But it's certainly complimentary, let's say, and what what Elon Musk does before that, I'm very curious to see. But then also, as you said, again, for listeners, Walter Isaacson tweeted, the text messages between Elon Musk and Mark Zuckerberg, where he's talking about, you know, let's have a cage match practice at your house or something, or I don't know exactly what which ones they were. But he is put it mixing himself in this whether he wants to or not. So to me, I don't think there's any kind of like journalistic duty to tweet the text messages. I think he's doing that is, as you said, more on the publicist side. I don't know, like, would you prefer reporters who had texts between Zuckerberg and Musk are powerful people in the tech industry, keep those close to the vest as opposed to share them to the public? I mean, I know if I got ahold of those text messages, I'm tweeting those immediately. Wait, but that that's any, any source, right? That's someone feeds you some bit of information. Why are they doing it? And should I share it? Or should I not? So it's the context around it? Yeah, it's the context. And clearly, Elon Musk is doing that to jab Mark Zuckerberg and using Walter Isaacson as his vessel to jab, jab Mark Zuckerberg. So, so I think he has to be more cognizant of that he spent however many hours and days and months with Elon Musk, he's got to know him a little better than that. I don't know, I would definitely be be sharing those messages. And with maybe with the context, like, all right, this, this is what is going on here. But, but it is interesting, we talked a little bit about, like, you know, is, is Isaacson going to write this book that makes people you know, that's effusive about Musk. And it's like, well, what do people want to read, in some ways, right? Like, is a book that's balanced or a book that's like a total hagiography of Elon Musk going to sell better and going to be appreciated by the public more. And it almost seems as if the public would rather read a book that's almost entirely positive about a business hero, like, you know, however much criticism we have of the media, like actually readers, they kind of want to read the book that like, this person, you know, face tough circumstances, and look at what all the good they did in the world, you know, it's one of the categories of books that you want to write is, you know, you want to have something that's aspirational to the reader that's going to make them part with the 30 bucks. Yeah, here's what you think. That's exactly an is Walter Isaacson functioning as a journalist here, or a biographer who's trying to sell books, and clearly, it'll be the latter. And yeah, I think that's but you know, exactly, exactly. You could be both. And I think a lot of people are. But I get to write the best sellers that are remembered. I mean, maybe that is the way that is, I agree. That's what people want to read. And it's kind of interesting to be discussing this in the same week that Michael Lewis's great book called a great in retrospect, who knows book called The Blind Side, which is about, you know, a boy who was adopted, and went on to play football and the adopted family like raised him and it was this very sentimental, beautiful story. And he went on to play for the Baltimore Ravens. And now he's come out and said, like, you know, they took him out of tough circumstances and the family now has, has tried to claim undue control over his finances and rewards from the story, due to a conservatorship versus adoption. And it's like this Michael Lewis story, this beautiful thread that he's, he's put together, it's starting to unravel. And it just goes to show you it's like, it's one of those things where I like, I do wonder if Michael Lewis actually and many authors just, you know, did he actually want to find the real story? Or was he content, telling the story that was, you know, almost, I think they call it in the business too good to check. It's very interesting. Yeah, I think it's tough, because that's where kind of like the things that border between fiction and non fictional narrative. It's tough, because like the blind side, and I say this, I'm a huge fan of Michael Lewis, and I really hope, like, more of this does not come to light. And it was a troubling one, because again, the the football player, Michael O'Hare, I believe, yeah, or yeah, he I mean, and it's kind of sad, too. Again, he came out, and has said the family tried to control his finances, but also that the movie and his portrayal while he was in the NFL actually negatively impacted him because it, you know, portrayed him and exaggerated, exaggerated his mental capacities. And of course, on the football field, that'll be something that's held against you. And that it's kind of like, like, it's sad that and I loved the movie. I loved the book. But remembering that it's happening while he is alive and while he's playing and, and, yeah, and it's a beautiful story. But a lot of the times the story is it's, Yeah, go ahead. it's too good to check. He came into the NFL and there were players that were wondering whether he could read the playbook because of the way that this movie portrayed him as this, like, you know, guy who'd been just sort of I mean, he's like, remember, like I was an All American before I came into the NFL. But because of this, the way that this story was told, and again, I think that there's a commercial aspect to it. You know, that's how it impacted his real life. So however, the one one thing I can say, again, Liar's Poker having worked on a trading floor for eight years, not during that time in the 80s with Salomon Brothers is feels and is as real as it can get about about capturing an industry and an environment. Yeah, look, we're not gonna come on here and say that Michael Lewis and Walter Isaacson are, you know, amateurs, like they obviously have skill and talent, but it is kind of interesting in terms of the commercialization and the way that these things are shaped, you know, the directions that the work go. And part of that's writer part of that story. But part of that is also audience, which doesn't get talked about very often. Speaking of Yeah, sorry. One thing on that, though, I think is interesting is, it's also a reminder. And again, the positive side of whether it's social media, whether it's just more people have a voice right now is in the past, it's very likely that, you know, the that book is written, and then Michael or his narrative never really makes it out into the world. Obviously, his was not from like a viral tweet or anything like that. But you do feel there is more accountability in general around these kind of things versus in the past. And I think that's a good thing. Yeah, I agree. And a lot of it happens on podcasts. I mean, I was watching or on a podcast. So we're doing good for the world, Ron John is what I'm trying to say. We're doing good for the world. There's speaking of doing good for the world Goldman Sachs. I'm just kidding. But you know, they're an interesting institution. And in the run up to this week show, they have a very interesting partnership with Apple, they've really been trying to make ways into the consumer world. And in the run up to this show, you dropped like a few links in our we have this collaborative doc that we like trying to drop stories in through the week to talk about them on this show. And you had dropped like a couple Goldman links. And I was like, I don't know, it's a little far afield. They do have that Apple partnership. But as I got deeper into the story, I was like, oh, yeah, this is definitely, you know, something that we need to talk about on the podcast. So not only is their partnership with Apple starting to struggle, but their CEO, David Solomon has has really come under fire inside the company. And it is kind of interesting as to like, why the folks at Goldman don't like him. It's not necessarily performance, but it's more just like his work on their reputation. I'm just gonna tee you up here, Ron, John, what about the Goldman News? And, you know, do you find interesting? And how do you think we should view its push into the consumer world, given that its program with Apple is so important, yet they've lost a lot of money there? Yeah, so okay, so to step back, remembering why and when Goldman Sachs got into consumer banking, I think is important here, because in 2016, they launched Marcus, which is the named after the founder, Marcus Goldman. And this was like the first really consumer facing product. This is the idea that Goldman Sachs before only for the ultra wealthy and exclusive, and only for, you know, Wall Street firms and banking and trading, suddenly is going to try to appeal to at least some larger portion of the entire like consumer industry. And it was it was at a time again, remember, fintech is probably like the promise of fintech mid 2010s, is it's going to change everything. It's going to be like in a revolutionary business, banking will change. So consumer banking will change. It's a wide open market. So they should get into it. It even from like a standpoint of recruiting, engineering talent to have these kind of like really digital first offerings. It was part of it. This is when they were losing lots and lots of talent to investment as I was sorry, to big tech companies, and wanting to make themselves appear more to be more technology forward. So there's so many different elements, then you throw in the the Apple card, which I'm assuming the discussions would have started around that time as well. You know, like having such a public facing partnership, having your logo imprinted on a credit card that's going to be in the wallet of probably a pretty good consumer profile for you again, still tech forward, like, probably good credit or reasonable credit. So that's like, and it started under Lloyd Blankfein, then David Solomon comes in and makes it really one of his kind of marquee strategic efforts. And he's like, we're going to consumer is going to be an area of growth. And sorry, and then another area I'd forgotten is this is also when because of post financial crisis regulatory reasons, banking, especially trading within an investment bank, starts to feel like it might become a little more restrained in terms of profit potential. So So where do you find growth, it's going to be in consumer and kind of digital forward consumer offerings. So all this is happening. And basically, like a lot, I think it's interesting how it reflects the lost promise of a lot of other fintech offerings and companies and unicorns. It's similar, they the same growing pains, the same inability to actually realize whatever potential was sold in the mid 2010s, then you have from like a pure loan book standing, they've been losing a ton of money on the loans that they actually underwrote on the consumer side and the credit that they issued. So all of this is happening. But why this story gets even better is you have David Solomon, you know, as the face of this push that he is the one and for those unfamiliar, David Solomon has a DJ name D soul. He's he flies around he DJs I think the chain smokers concert during COVID was one of the one of the one of the big and most controversial ones, but he has he's created this persona for him in front of everyone totally publicly. And it's been okay, but it's one of those that when it's working, and that's good for you. The moment any kind of hiccup comes around, you can imagine everyone is going to be coming for you. And that's what we started to see there was an insane profile in New York magazine, I recommend everyone reads, it's like, it's a long one, but it's so good. And you can see it happening that all these kind of intersecting trends are coming together right now. Again, like the lost promise of FinTech, people who are executives who over the last few years might have kind of gone a little too far on the personal branding and promotions and being the cool guy side. And now it needing to be just getting back to business. So I think this this is one definitely to watch. Right. So just going over the story. So they basically needed to get into consumer stuff, because profits on investment banking side might not have been what they would have been pre regulation. And they also saw an opportunity with FinTech like thinking about like maybe like a SoFi as a competition. So that's why they decided to expand into consumer. Yeah, exactly. Think SoFi, Betterment, Wealthfront, that's one entire area. So imagine why did they need that though? That's the thing. Like they are like this prestigious investment bank, they have all the big clients, they probably could still make a ton of money. Like even the stories talk about how like Goldman can make money when it needs to make money. No, so what what is the Why does a luxury fashion house ever have like a more mass market sub brand? Right? That's true. But it's not working out for them, though. So what do you think? What do you think has been the cause for them? I mean, they've lost a billion dollars on the Apple card alone. Yeah, no, no, I think again, I think it's not working out. But but if you go back to 2016, I think or probably 2014, when you can imagine they would have been pitching this strategically internally, this would have been the greatest pitch deck in the world for a senior executive who wants to kind of move their way up and impress people and then David, even Lloyd Blankfein at the time and then David Solomon, like this, it's a dream pitch, right? It just doesn't quite work out. And then you throw in the personal side of jet setting around and DJing with the chain smokers. It's not only the DJing. He's just a jerk internally, according to this profile. I mean, there was this one line that I pulled out where he didn't like what was happening in with, you know, some of his employees. And he goes, I can't wait for the markets to turn so I can start firing you. And it's like, yeah, I guess that's the personality of finance in some ways, not all finance houses, but Goldman in particular seems like one where that's tolerated. And yeah, I guess there's there's blowback. So So do you think that the things that have caused the blowback have been that like fintech is harder than they expected? And then the market did turn? Yeah, I think it's exactly those two things. fintech is harder than expected. And the market turned. And I think and actually on the subject of he's just kind of a jerk. I kind of I kind of smiled when I was reading a lot of the the kind of over the top comments made by him or things that are being attributed to him about being a bit abrasive, let's say, is because I mean, this is investment banking. This is Wall Street. This is like, that's just the culture that feels it's been, you know, over a decade that I have not been directly working in it. But like, I mean, that stuff seems pretty tame or peaceful, compared to a lot of things you would see to but but it's a reminder you see, are you able to talk about it? Oh, no, I mean, just being berated in front of people and yelled at or that happened to you? Oh, yeah, yeah. That's easy. Yeah. In front of the whole floor, people your boss screaming at you, you're then those are the bad ones. There's good ones too. So I, one of my bosses really loved them and still talk to them regularly now. But that stuff, it's pretty it was it was honestly very normal and in a weird way, like to talk about it again, 2023 trading floor culture, I imagine it's a bit difficult, different now. But but I think on that side, but again, and David Solomon is kind of like falling into the trap of being allowed to be an asshole for a bit. You're allowed to until you stop performing. Right. And then the knives come out and you can see it being a nice guy would, you know, what a safe time on this one, obviously just the numbers. But the interesting thing is, I mean, I think it's worth talking about this apple card in particular. So I'm curious why you think they've lost so much money on it. I mean, more than a billion dollars. And then there was another like really interesting part of the story where they talk about like, who their customers are now. So they went from like the highest and type customers. And this is, you know, what's happened recently talking about what went wrong. Straight from the New York mag story. Rising interest rates caused defaults on Apple cards to spike more than a quarter of its credit card borrow borrowers had FICO scores under 660. Most of its personal loan customers were using the money to pay down other debt. Goldman Sachs suddenly had millions of customers who not only weren't as rich as the banks long time, long time clients, but were struggling to make ends meet. And I think that's sort of the crux of it, right? It just like talks exactly about how tough it is for a long time bank to come in and start to take on some of like the, you know, down the line risk that fintech companies are willing to take on to grow. Yeah, so, as you said, it's actually estimated one to $3 billion over the life of the partnership. It's just, it's bad issuance of credit, essentially, it's like, it's not it's not. And I could also I could almost imagine because remember, in the mid 2010s, like, there was that promise that you should be able to using data evaluation, like, understand a credit score and the credit worthiness of an individual consumer at a far more granular level than a FICO score, like, you know, using there's even stuff I remember where it was like, using social media, we will be able to issue credit based on, on like who someone is and what they're talking about. And, you know, like, imagine you're like, posting about your vacation somewhere, and that makes you seem more wealthy. So that's going to give you more credit access to credit. So, so there, there is a lot of promise around being able to issue credit more rapidly to a wider group of customers, but they did not execute well, I think that was the actual quote, I read in one of the articles where it was we did not execute well, right, I think they're, they're paying for it now. And now they're thinking about passing that partnership on to American Express. So quite interesting. All right, let's take a break, come back, talk a little bit about the suit, these new studies about social media, Ron John Roy is here with us back from Taiwan, off of vacation. And now it's time for us to get back into the swing of things. So thanks, everybody who's been listening here. And we'll be back after this with a discussion of those two topics. An incredibly deep voice doesn't sell B2B and advertising on the wrong platform doesn't sell B2B either. That's why if you're a B2B marketer, you should use LinkedIn ads. With LinkedIn ads, you'll be able to target over 70 million decision makers all in one place. No deep voice required. Get started today and see why LinkedIn is the place to be to be. We'll even give you a $100 credit on your next campaign. Go to linkedin.com slash platform to claim your credit terms and conditions apply. And we're back here on Big Technology podcast, Ron John Roy is back from vacation, we're going to talk a little bit in this section about what he saw in Taiwan. Actually, maybe we can just start with that. Ron, do you have any like reflections of, of, like the Taiwanese tech culture, anything that you saw just by osmosis there? And then we can move on to these interesting studies about social media algorithms. Yeah, the the first one, I got two that like really stuck out. First one, a lot of places did not take credit cards as in Visa or Amex Visa MasterCard or Amex. But it wasn't cash. It was all line pay. Now we chat pay but line for those unfamiliar another messaging app, messaging app slash super app that Elon Musk wants to create. And for me, it was it was pretty fascinating to watch because everything even like, like roadside vendors and stuff, everyone took line pay. You know, like everyone the call to action is connect with us online. So seeing the behavior of the super app in her, you know, in person, and it makes you have more faith in Elan's plan. No. I tried. Yeah. The other one, this was one of my favorite. So in terms of delivery, they had something called Lala move, which apparently is in the Philippines and through Southeast Asia, East Asia. And so we were we were out at dinner, and with one of my wife's friends and his wife was at home. So they literally order food to go. And then you can order is essentially a courier service. But it's very popular. So rather than having direct relationships with the restaurant even necessarily or the restaurant isn't even on the platform, but just calls couriers. So people because I believe I mean, it was explained to me, again, because there is an abundance of lower cost labor and people and you can even order like on a moped, on a car, on a truck. So yeah, so the whole delivery infrastructure, it's like, it's something that in the US I never think about it. I know I mean, Uber had has or had Uber courier like moving stuff around outside of the confines of a food delivery app or a car service, but actually being able to just have someone move around something for you instantly. It was something I saw over and over again, that everyone was using this Lala move. Wow, that's interesting. Could that ever work in the US? I think I don't know. I mean, again, Uber, I know they had the courier service it couriers I've always still thought of more for like work. When someone had like some document that has to move around or some box that has to go from one place to another immediately, but but it could make sense. Again, you're you're the restaurant rather than necessarily contracting to a food delivery startup, but just that same startup that brought them there, like, you know, from a vendor supplier stuff in the morning, could also be delivering stuff to customers when it works out. Yeah, I was desperately looking for one of those services in New York recently where I had like left my car in a mechanic in Long Island. It was old folks, but it died. And I was I sold it to like one of those like parts for hire people. And the repair people because I didn't have any more repairs just stopped picking up the phone. And I had to leave the title in the car before the night before the repair people came came in. So I ended up going on my own, you know, going out there on my own taking the plates off, leaving the title in the in the driver's seat. And it was like 10pm was like probably the night before I flew to San Francisco. And because they were not ready for me because they stopped answering my answering my calls. And I needed to get that done that night. I'm in the parking lot doing the thing. And they called the cops like they were monitoring it like, remotely. I'm like, trying to get this car off my hands. And next thing I know, there's like some some speaker saying you are trespassing, the police have been called. I'm like, this is my car. So hold on. So just to confirm, you would like to pass that off to no to only for the first half of the story to career. Yeah, but you know what I think because I could only get out there at night. But if I were to be able to send a courier during the day, there wouldn't have been a problem. Okay, no, no, that's fair. And actually, yeah, that exactly. Here is the title to the car. Go and just drop it off. Yeah, yeah. I mean, I guess a lot of money for that also. Because like, you know, I mean, it just had to be done in the moment. And it would have saved me a drive out to Long Island to do it. So I guess yeah, I mean, I guess I had to drive out to Long Island almost got arrested for taking the plates off my own trespassing. It would have been really awful. So social media algorithms, social media algorithms. Social media algorithms are having their moment in the sun. Recently, there have been four academic institutions that studied the role of social media algorithms, and features of the feed. And they published their findings in late July, and, and they've continued to be discussed because they're actually like, fairly large studies. And they're covered in places like the New York Times. I don't know, I didn't make too much about them because they came off of like, there wasn't like universal access to Facebook, it was just like Facebook handed over the data. But some of their other conclusions have been quite interesting, including that, that when you were to use a plain reverse chronological feed versus an algorithm, it doesn't increase polarization, among many other things. And I'm curious, like, it seemed overwhelmingly positive for for Facebook and companies that use algorithms to figure to filter news. But I was also just like, kind of caveat here, right? Because look at the source, what was your perspective on on these studies? What did you learn? Two things. One, definitely caveat here, because the studies were funded by Facebook. In 2019, they committed $20 million to work with the NRC at the University of Chicago non partisan research organization. But they did not pay the researchers directly, but they did fund the overall research. And they also had control to exactly what access what data the researchers had access to. So again, yeah, this is still essentially a Facebook study that ends up, at least from a PR standpoint, they leaned heavily into the idea that our algorithms do not affect polarization at all. And this proves everything. I disagree with that. And I dug into this a lot, because actually, one of the things I strongly believe in I remember a year a couple of years ago, in like 2019, one of the earliest margins post was my five point plan to fix social media. And one of them and it's number one must has to just buy Twitter. Is that on? That was it? That was what I said from the beginning. Well done. And Mark Zuckerberg needs to copy threads. It needs to launch threads. Very precious. We're there. Nailed it. So, so to me, the idea of a reverse chronological feed in stead of an algorithm, I think is one of the most important things that needs to be available. It needs to be very potentially like my I half joked, but you the default should be reverse chronological. And if you want to opt into algorithms, you should have like a warning label like a pack of cigarettes is what I've said, is that the idea is like you just have to be made aware that what is being fed to you in an algorithm is going to be biased towards the more extreme because that's how algorithms work. This study and we can definitely get more into that. And I think tick tock became this kind of like logical endpoint of what a pure algorithm looks algorithmic content looks like when there's no social graph, no anything else. So the study found that when you actually go into a reverse chronological feed that supposedly it does not add to polarization. In fact, it it shows you less political content is what the study found. The thing that was interesting to me, but it also is it also found that it shows you less extreme content, which is obvious, we all know the more viral something is, the more an algorithm will select and show it. It's not like that's the thing for me around a lot of this is it's not you know, rocket science. It's, of course, things that are viral or pushed by an algorithm are going to be more extreme. So so to me, Facebook clearly, and like Nick Clegg was everywhere saying this shows that we did not actually, you know, increase polarization during the election. But it took it took like a very small portion of time right before one month before the November 2020 election, when feeds are very heavily political anyways, and then still found that reverse chronological feed will show you less extreme content. It's just worse, though, like even I think about threads, like, you know, I'm on the for you. And there's also they just released the following tab a few weeks ago, and I never want the following. It's terrible. It's terrible. And here's why that's so your plan is to just break social media. I mean, there's got to be some algorithmic filtering. No, no, it has to be a record. So two things, I still believe, but no, and don't get me wrong, I say this not necessarily tongue in cheek, it would definitely hamper or not maybe destroy a lot of businesses, as tick tock would definitely be the most vulnerable. And actually, in Europe, tick tock is now going to have to show as part of the European Digital Services Act, a non algorithmic option. So they've already announced what they're going to do is instead of but remember, it doesn't have to just be follower based, which is threads. The problem is, their reverse chronological feed is only followers, right? It's or sorry, the and call it not reverse chronological, but the non algorithmic feed is followers. I will say I went on there the other day. No one I know has threaded in days, I think I thought I saw when was your last thread? Maybe like, I can't recall, but not definitely. No, come on. Are you doing it just so you can come on this podcast and tell me that you're still threading? That is the number one reason. But I have a very I will I think I used to have an active community on thread. No, my threaded five days ago. Oh my god, yeah, I really felt the wagon felt the wagon on threads. Start threading again, man. No, I think that it seems like that network is going to go through very long struggle, and may never actually get back to what it was that first that first week, it feels dead. Right now. I'm just gonna say it. It I'm admitting it. Thank you. Thank you. Another thing I gotta admit, by the way, I'm going to get back to this algorithm thing is this week, a self driving cruise drove directly into wet concrete in San Francisco. This is from the New York Times. On Tuesday in San Francisco, a driverless car somehow drove into a city part paving project and got stuck in wet concrete. The mishap involved the cruise vehicle. According to city officials who said it was not clear how this car ended up in the concrete. I know I've been very effusive on self driving cars. I'm not going to be one sided about it like Wilder Isaacson will be about Elon Musk. I admit it ain't perfect yet. Okay, just gotta say that. I feel like just for for, you know, full disclosure reasons. Yes, these cars do sometimes drive into the concrete. But but back to our story about wet concrete, it was wet paper. They said they they had like recovered the car and it wasn't clear like what, what do they leave the wheels in there? They have to pay anyway to figure that out. But but going back to our discussion of algorithms, I think the thing that's been most interesting about this that's really picked up speed, whether it's Facebook itself, whether it's Musk inside Twitter, or even advocates like Julie Agwin, who wrote about it this week is that there's budding momentum for people to be able to sort of choose their own algorithm as opposed to just go reverse cron or algorithm. You should be able to choose a flavor of your algorithm, you could choose like, heavily filtered, lightly filtered, outrage algorithm, chill algorithm, funny algorithm, and just allow that to filter your feed based off of your preferences. I think we're eventually going to get there. It seems like that is definitely like the next stage of social media is just like sort of pick your flavor and then run with it. And I'm curious if you think that you know, you pointed out that article to me that Julia one. Is that like the the ideal and most pragmatic outcome in one? Yeah, I really think and I hope that's where we move to because again, when I'm saying reverse chronological should be the default, I'm not saying that should be the only option. I'm saying that the least extreme and outrageous one should be where everyone starts. And then you should be able to go where you want to or like, like, rather than passively being fed and told what to look at, you should have to define and do a little bit of work for yourself. And the problem is, that friction will obviously change the user growth and like engagement for any platform, right? So obviously, and I'm not saying from a business standpoint, it's the best way to approach this, but from an actual informational health standpoint, like having to just say this is what I want to do. So if you want to get that shit outrageous content, you have you should have that right, you should have to click this is what I want. That's my threads. That's why I die. It was just too crazy for me. Give me the absolute craziest stuff out there. I want only lies only lies. And then I should be able to choose that but I should have to choose that and not just be fed it because of what I've been profiled at or what I'm clicking on. Love that. All right, Ron, john, feels good to be back. Feels got a lot to talk about next week. We're gonna have a whole segment on whether generative AI is a dud. And speaking of which Colin Murdoch, the chief business officer of deep mind who does not believe it's a dud is going to be on the podcast on Wednesday to talk a little bit about Google's efforts in this area. So we managed to do a whole show without any gen AI this week. I think that's great. And we'll get back to it next week really interrogating, you know, how far it's come and what the opportunity is and where it lacks, because there have been, you know, a couple months or almost a year and now some areas that have definitely emerged and some concerns that have definitely emerged amid the hype. So we will talk about that then. Ron, welcome back. It's really fun to have you here. generative AI. Yeah, and more. And maybe maybe another Long Island trespassing story. Well, I'm in Seattle now. So okay, so that's where the trespassing will take place. I'm planning to do zero crimes for the next week or so as I make my way back to San Francisco. And then eventually to New York, but no promises. So if, if I end up taking the plates off another card ain't going to be for for for good reasons, folks, that I promise you, but I will not be doing that. I left my screwdriver back in the apartment. Thank you, Ron, john. Thanks. Thanks, everybody for listening. Pleasure to have you back Ron, john will be doing this again every week with our Friday news recap show. And then tune in on Wednesdays for the flagship interview show. Again, Colin Murdoch from deep mind, our first deep mind interview yet is going to be on Wednesday. Don't miss it. Thanks everybody for listening. We will see you next time on big technology podcasts.
# """

In [ ]:
#@title Summary of Podcast
# determine the number of tokens in the text
enc = tiktoken.encoding_for_model("gpt-3.5-turbo-16k")
print ("Number of tokens in input prompt ", len(enc.encode(podcast_transcript)))

Number of tokens in input prompt  10219


In [ ]:
pattern = r'\b[A-Z][a-z]+ [A-Z][a-z]+\b'

# Find all occurrences of the pattern in the text
matches = re.findall(pattern, podcast_transcript)

# Print the extracted names
for match in matches:
    print(match)

Goldman Sachs
Ryan Ruzlanski
The Path
Big Technology
Podcast Friday
Ron Johnroy
Ron John
Elon Musk
Ron John
Elon Musk
Mark Zuckerberg
Walter Isaacson
Elon Musk
Mark Zuckerberg
Walter Isaacson
Elon Musk
Mark Zuckerberg
New York
Elon Musk
Elon Musk
Elon Musk
Felix Salmon
Elon Musk
Walter Isaacson
Steve Jobs
Walter Isaacson
Linda Yaccarino
But Elon
But Walter
Walter Isaacson
Walter Isaacson
Elon Musk
Steve Jobs
Elon Musk
Walter Isaacson
Elon Musk
Mark Zuckerberg
Elon Musk
Mark Zuckerberg
Walter Isaacson
Mark Zuckerberg
Elon Musk
Elon Musk
Walter Isaacson
Michael Lewis
The Blind
Baltimore Ravens
Michael Lewis
Michael Lewis
Michael Lewis
All American
Salomon Brothers
Michael Lewis
Walter Isaacson
Ron John
Goldman Sachs
David Solomon
Goldman News
Goldman Sachs
Marcus Goldman
Goldman Sachs
Wall Street
Lloyd Blankfein
David Solomon
So So
David Solomon
David Solomon
New York
Lloyd Blankfein
David Solomon
So So
Wall Street
David Solomon
New York
Goldman Sachs
American Express
Ron John
Big Techno

In [ ]:
#@title attempt to find podcast speaker name


# Load a language model for named entity recognition
#The spacy library is used for named entity recognition to validate speaker names.
nlp = spacy.load("en_core_web_sm")




# Tokenize the transcript
encoded_transcript = enc.encode(podcast_transcript)

# Convert encoded transcript back to text
decoded_transcript = enc.decode(encoded_transcript)

# Split the transcript into sentences or paragraphs
sentences = decoded_transcript.split('.')  # Modify based on the actual format

# Initialize a list to store speaker information
speakers = []
# The validate_speaker_names function applies NER and returns valid speaker names.
# The speakers list stores identified speakers, which are later validated.

# # Define regular expression patterns for speaker identification
# speaker_pattern = re.compile(r'(Host|Guest|Speaker|I\'m):', re.IGNORECASE)

# # Process each sentence to identify speakers
# for sentence in sentences:
#     if speaker_pattern.search(sentence):
#         matches = speaker_pattern.findall(sentence)
#         for match in matches:
#             speaker_name = match.capitalize()  # Assuming speakers are identified consistently
#             speakers.append(speaker_name)

# # Apply named entity recognition for speaker names validation
# def validate_speaker_names(text):
#     doc = nlp(text)
#     valid_speakers = []
#     for ent in doc.ents:
#         if ent.label_ == "PERSON":
#             valid_speakers.append(ent.text)
#     return valid_speakers

# # Validate and clean the list of speaker names
# valid_speakers = validate_speaker_names(" ".join(speakers))

#valid_speakers


In [ ]:
pattern = r'(?:I\'m|I am|Here with us today is)\s+([A-Z][a-z]+ [A-Z][a-z]+)'
matches = re.findall(pattern, podcast_transcript)

speakers = "The speakers in this podcast are"
print("The podcast speakers are")
for match in matches:
    print(match)
    speakers = speakers + ", "+ match

The podcast speakers are
Ryan Ruzlanski
Ron Johnroy


In [ ]:
print(speakers)

The speakers in this podcast are, Ryan Ruzlanski, Ron Johnroy


In [ ]:
#@title setting the prompt

instructPrompt = """
User
You are a technology enthusiast and love to listen to technology podcasts. You have limited time in your busy schedule so you prefer to listen to a summary of the podcast. Please help write a concise summary of this podcast, please highlight on all the interesting aspects discussed in the podcast and mention name of speakers.
The transcript of the podcast is provided below:
"""

request = speakers + instructPrompt + podcast_transcript


In [ ]:
chatOutput = openai.ChatCompletion.create(model="gpt-3.5-turbo-16k",
                                            messages=[{"role": "system", "content": "You are a helpful assistant."},
                                                      {"role": "user", "content": request}
                                                      ]
                                            )

In [ ]:
podcastSummary = chatOutput.choices[0].message.content
podcastSummary

"In this podcast episode, the hosts discuss various topics, including the failed cage match between Elon Musk and Mark Zuckerberg, Goldman Sachs' struggles with its partnership with Apple, and the impact of social media algorithms. They talk about how Musk's reputation and behavior are integral to his success, and how his actions on social media can both benefit and harm him. They also debate the effectiveness of Goldman Sachs' move into consumer banking and the challenges they face. Lastly, they discuss recent studies on social media algorithms funded by Facebook, which claim that algorithms do not increase polarization. The hosts express skepticism about these findings and argue that users should have the ability to choose their own algorithmic preferences on social media platforms."

In [ ]:
chatOutput2 = openai.ChatCompletion.create(model="gpt-3.5-turbo-16k",
                                            messages=[{"role": "system", "content": "You are a technology enthusiast."},
                                                      {"role": "user", "content": request}
                                                      ]
                                            )
summary2 = chatOutput2.choices[0].message.content
summary2

'Goldman Sachs is facing challenges as its partnership with Apple is taking off, and there is growing skepticism about the impact of social media algorithms. In a recent podcast, the CEO of LinkedIn, Ryan Ruzlanski, and Ron Johnroy discussed these issues. Goldman Sachs ventured into consumer banking in 2016 with the launch of Marcus, an attempt to tap into the growing fintech industry. However, the partnership with Apple has been a costly endeavor, with Goldman Sachs losing over a billion dollars on its Apple Card program. Additionally, the CEO of Goldman Sachs, David Solomon, has come under fire internally for his handling of the partnership and his reputation management. While the studies on social media algorithms funded by Facebook have sparked debate, they suggest that a reverse chronological feed does not lead to increased polarization. However, it is important to consider that the studies were funded by Facebook and had limitations in terms of access to data. The discussion arou

In [ ]:
tts = gTTS(summary2)

# Save the generated speech as an audio file
tts.save("summary_output.mp3")
ipd.Audio("summary_output.mp3")


In [ ]:
#@title looking at the transcript for guest and looking them up in Wikipedia
request = podcast_transcript[:5000]
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
print ("Number of tokens in input prompt ", len(enc.encode(request)))

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": request}],
    functions=[
    {
        "name": "get_podcast_guest_information",
        "description": "ADD_THE_FUNCTION_DESCRIPTION_HERE",
        "parameters": {
            "type": "object",
            "properties": {
                "guest_name": {
                    "type": "string",
                    "description": "ADD_THE_VARIABLE_DESCRIPTION_HERE",
                },
                "unit": {"type": "string"},
            },
            "required": ["guest_name"],
        },
    }
    ],
    function_call={"name": "get_podcast_guest_information"}
    )

Number of tokens in input prompt  1096


In [ ]:
completion

<OpenAIObject chat.completion id=chatcmpl-7pBcKrucxbLk1cmyczc5CGQ9Qt2vb at 0x78ba99a17e70> JSON: {
  "id": "chatcmpl-7pBcKrucxbLk1cmyczc5CGQ9Qt2vb",
  "object": "chat.completion",
  "created": 1692433740,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_podcast_guest_information",
          "arguments": "{\n  \"guest_name\": \"Ryan Ruzlanski\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1165,
    "completion_tokens": 15,
    "total_tokens": 1180
  }
}

In [ ]:
#parsing the json

podcast_guest = ""
response_message = completion["choices"][0]["message"]
if response_message.get("function_call"):
  function_name = response_message["function_call"]["name"]
  function_args = json.loads(response_message["function_call"]["arguments"])
  podcast_guest=function_args.get("guest_name")

print ("Podcast Guest is ", podcast_guest)

Podcast Guest is  Ryan Ruzlanski


In [ ]:
#testing Wikipedia search
input = wikipedia.page("Elon Musk", auto_suggest=False)
podcast_guest_info = input.summary
print (podcast_guest_info)

Elon Reeve Musk ( EE-lon; born June 28, 1971) is a business magnate and investor. Musk is the founder, chairman, CEO and chief technology officer of SpaceX,  angel investor, CEO and product architect of Tesla, Inc., owner, chairman and CTO of X Corp., founder of the Boring Company, a co-founder of Neuralink and OpenAI, and the president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$217 billion as of August 2023, according to the Bloomberg Billionaires Index, and $219 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University in Kingston, Ontario. Musk later transferred to the University of Pennsylvania, and received bachelor's degrees in economics and physics there. He mo

In [ ]:
try:
    input = wikipedia.page(podcast_guest, auto_suggest=False)
    podcast_guest_info = input.summary
    print(podcast_guest_info)
except wikipedia.exceptions.DisambiguationError as e:
    print(f"No match found for '{podcast_guest}'. Suggestions:", e.options)
except wikipedia.exceptions.PageError:
    print(f"No match found for '{podcast_guest}'.")

No match found for 'Ryan Ruzlanski'.


In [ ]:
#@title podcast highlights
instructPrompt2 = """
With the time stamps mention the key points discussed in the podcast as a list.
"""

request = instructPrompt2 + podcast_transcript

In [ ]:
chatOutput = openai.ChatCompletion.create(model="gpt-3.5-turbo-16k",
                                            messages=[{"role": "system", "content": "You are a helpful assistant."},
                                                      {"role": "user", "content": request}
                                                      ]
                                            )
podcastHighlights = chatOutput.choices[0].message.content
podcastHighlights

"Key points discussed in the podcast:\n\n1. The Goldman Sachs partnership with Apple is struggling, and CEO David Solomon has come under fire within the company.\n2. Recent studies funded by Facebook suggest that social media algorithms do not increase polarization, but critics argue that the studies should be viewed with caution due to Facebook's involvement.\n3. There is growing momentum for allowing users to choose their own algorithm preferences, such as heavily filtered, lightly filtered, or based on specific interests.\n4. Consideration is given to the potential benefits and drawbacks of a reverse chronological feed versus an algorithmic feed.\n5. The upcoming interview with Colin Murdoch from DeepMind will explore the progress and concerns surrounding generative AI."

In [ ]:
tts = gTTS(podcastHighlights)

# Save the generated speech as an audio file
tts.save("highlight_output.mp3")
ipd.Audio("highlight_output.mp3")

#BACKEND

###library installations & imports

In [ ]:
#@title installing Modal
!pip install modal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.8/284.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.8 MB/s eta 0:00:00
  Created wheel for grpclib: filename=grpclib-0.4.3-py3-none-any.whl size=77058 sha256=7bbe14457d352ef02283a9a6ad69052b2889f599c11377911ac91d494328b871
  Stored in directory: /root/.cache/pip/wheels/e0/49/4d/31aa7f3f43e9833b8e79204457b404ad4434fc2278a32fa9f1
Successfully built grpclib
  Attempting uninst

#FRONTEND